In [4]:
pip install tweet-preprocessor


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
#load Data
import numpy as np 
import pandas as pd
import preprocessor as p
import matplotlib.pyplot as plt
import seaborn as sns
import re, string, unicodedata
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords
from tensorflow import keras

In [ ]:
tweets = pd.read_csv('tweets.csv',encoding='utf8')
tweets



In [ ]:
sns.countplot('target', data=tweets)
plt.title('Perbandingan Data Setiap Kelas')

In [ ]:
tweets['target'].value_counts()

In [ ]:
#preprocessing
def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

tweets['textprep'] = tweets.apply(preprocess_tweet, axis=1)

In [ ]:
tweets

In [ ]:
def preprocess_data(data):
 #menghapus angka
 data = data.astype(str).str.replace('\d+', '', regex=True)
 #menjadikan lowercase
 lower_text = data.str.lower()
 #lemmatize
 lemmatizer = nltk.stem.WordNetLemmatizer()
 #tokenize
 w_tokenizer =  TweetTokenizer()

 def lemmatize_text(text):
  return [(lemmatizer.lemmatize(w)) for w \
                       in w_tokenizer.tokenize((text))]
 def remove_punctuation(words):
  new_words = []
  for word in words:
      new_word = re.sub(r'[^\w\s]', '', (word))
      if new_word != '':
         new_words.append(new_word)
  return new_words

 words = lower_text.apply(lemmatize_text)
 words = words.apply(remove_punctuation)
 return pd.DataFrame(words)

prep_tweets = preprocess_data(tweets['textprep'])
tweets['textprep2'] = prep_tweets

In [ ]:
tweets

In [ ]:
stop_words = set(stopwords.words('english'))
tweets['textprep2'] = tweets['textprep2'].apply(lambda x: [item for item in \
                                    x if item not in stop_words])

In [ ]:
tweets

In [ ]:
#join tokenize data
tweets['textprep_join'] = tweets['textprep2'].apply(lambda x: ' '.join(x))
tweets